In [1]:
import pandas as pd
import numpy as np


In [2]:
df_loaded = pd.read_csv('./clean_data/clean_data.csv')
df_loaded.drop('Unnamed: 0', axis=1, inplace=True)

In [3]:
df_loaded.head()

,strDrink,strCategory,strGlass,strIngredients,Alc_type,Basic_taste,strInstructions,strMeasures,Value,MeasureName,Value_numeric,Value_ml,Value_gr,Garnish_amount,Garnish_type,MeasureName_copy
0,'57 Chevy with a White License Plate,Cocktail,Highball glass,Creme De Cacao White,Creamy Liqueur,NaN,1. Fill a rocks glass with ice 2.add white cre...,1 oz white,1,oz,1.0,30.0,NaN,NaN,NaN,white
1,1-900-FUK-MEUP,Shot,Old-fashioned glass,Absolut Kurant,Vodka,NaN,Shake ingredients in a mixing tin filled with ...,1/2 oz,1/2,oz,0.5,15.0,NaN,NaN,NaN,oz
2,110 in the shade,Beer,Beer Glass,Lager,Beer,NaN,Drop shooter in glass. Fill with beer,16 oz,16,oz,16.0,480.0,NaN,NaN,NaN,oz
3,151 Florida Bushwacker,Milk / Float / Shake,Beer mug,Malibu Rum,Rum,NaN,Combine all ingredients. Blend until smooth. G...,1/2 oz,1/2,oz,0.5,15.0,NaN,NaN,NaN,oz
4,155 Belmont,Cocktail,White wine glass,Dark Rum,Rum,NaN,Blend with ice. Serve in a wine glass. Garnish...,1 shot,1,shot,1.0,25.0,NaN,NaN,NaN,shot


In [4]:
df = df_loaded.drop('MeasureName_copy', axis=1)

In [5]:
df.drop(df.loc[(df['Value_ml'].isnull()) & (df['Value_gr'].isnull()) & (df['Garnish_amount'].isnull())].index,
                                                                                                           axis=0, inplace=True)

In [6]:
# Create and fill new column with correct measure names

df['Measure'] = np.nan
df['Measure'] = df['Measure'].astype('object')
df['Measure'] = np.where((~df['Value_ml'].isnull()), 'ml', 'gr')
df['Measure'] = np.where(~df['Garnish_amount'].isnull(), 'garnish', df['Measure'])
df['Measure'].unique()

array(['ml', 'gr', 'garnish'], dtype=object)

In [7]:
# Create and fill new column with correct amounts of each ingredient

df['Volume'] = np.nan
df['Volume'] = df['Volume'].astype('object')
df['Volume'] = np.where(df['Measure'] == 'ml', df['Value_ml'], df['Value_gr'])
df['Volume'] = np.where(df['Measure'] == 'garnish', (df['Garnish_amount']+' '+df['Garnish_type']), df['Volume'])
df.head()

,strDrink,strCategory,strGlass,strIngredients,Alc_type,Basic_taste,strInstructions,strMeasures,Value,MeasureName,Value_numeric,Value_ml,Value_gr,Garnish_amount,Garnish_type,Measure,Volume
0,'57 Chevy with a White License Plate,Cocktail,Highball glass,Creme De Cacao White,Creamy Liqueur,NaN,1. Fill a rocks glass with ice 2.add white cre...,1 oz white,1,oz,1.0,30.0,NaN,NaN,NaN,ml,30
1,1-900-FUK-MEUP,Shot,Old-fashioned glass,Absolut Kurant,Vodka,NaN,Shake ingredients in a mixing tin filled with ...,1/2 oz,1/2,oz,0.5,15.0,NaN,NaN,NaN,ml,15
2,110 in the shade,Beer,Beer Glass,Lager,Beer,NaN,Drop shooter in glass. Fill with beer,16 oz,16,oz,16.0,480.0,NaN,NaN,NaN,ml,480
3,151 Florida Bushwacker,Milk / Float / Shake,Beer mug,Malibu Rum,Rum,NaN,Combine all ingredients. Blend until smooth. G...,1/2 oz,1/2,oz,0.5,15.0,NaN,NaN,NaN,ml,15
4,155 Belmont,Cocktail,White wine glass,Dark Rum,Rum,NaN,Blend with ice. Serve in a wine glass. Garnish...,1 shot,1,shot,1.0,25.0,NaN,NaN,NaN,ml,25


In [8]:
# Combine ingredient, volume and measure in one sentence

def combine(line, ingr_col1, ingr_col2=None):
    if ingr_col2:
        lst1 = [line[ingr_col1], line[ingr_col2], line['Volume'], line['Measure']]
    else:
        lst1 = [line[ingr_col1], line['Volume'], line['Measure']]
    lst2 = [str(i) for i in lst1 if str(i) != 'nan']
    return ' '.join(lst2)

    
# combine(df.iloc[1], 'Alc_type')      # test

In [9]:
# One column - alcohol type and basic taste
# Another column - ingredients as they are
# Third column - alcohol type and non-alcoholic ingredient

df['Sentence_type'] = df.apply(lambda x: combine(x, 'Alc_type', 'Basic_taste'), axis=1)
df['Sentence_ingr'] = df.apply(lambda x: combine(x, 'strIngredients'), axis=1)
df['Sentence_type_ingr'] = np.where(df['Alc_type'].isnull(),
                                    df.apply(lambda x: combine(x, 'strIngredients'), axis=1),
                                    df.apply(lambda x: combine(x, 'Alc_type'), axis=1))

In [10]:
df.head()

,strDrink,strCategory,strGlass,strIngredients,Alc_type,Basic_taste,strInstructions,strMeasures,Value,MeasureName,Value_numeric,Value_ml,Value_gr,Garnish_amount,Garnish_type,Measure,Volume,Sentence_type,Sentence_ingr,Sentence_type_ingr
0,'57 Chevy with a White License Plate,Cocktail,Highball glass,Creme De Cacao White,Creamy Liqueur,NaN,1. Fill a rocks glass with ice 2.add white cre...,1 oz white,1,oz,1.0,30.0,NaN,NaN,NaN,ml,30,Creamy Liqueur 30.0 ml,Creme De Cacao White 30.0 ml,Creamy Liqueur 30.0 ml
1,1-900-FUK-MEUP,Shot,Old-fashioned glass,Absolut Kurant,Vodka,NaN,Shake ingredients in a mixing tin filled with ...,1/2 oz,1/2,oz,0.5,15.0,NaN,NaN,NaN,ml,15,Vodka 15.0 ml,Absolut Kurant 15.0 ml,Vodka 15.0 ml
2,110 in the shade,Beer,Beer Glass,Lager,Beer,NaN,Drop shooter in glass. Fill with beer,16 oz,16,oz,16.0,480.0,NaN,NaN,NaN,ml,480,Beer 480.0 ml,Lager 480.0 ml,Beer 480.0 ml
3,151 Florida Bushwacker,Milk / Float / Shake,Beer mug,Malibu Rum,Rum,NaN,Combine all ingredients. Blend until smooth. G...,1/2 oz,1/2,oz,0.5,15.0,NaN,NaN,NaN,ml,15,Rum 15.0 ml,Malibu Rum 15.0 ml,Rum 15.0 ml
4,155 Belmont,Cocktail,White wine glass,Dark Rum,Rum,NaN,Blend with ice. Serve in a wine glass. Garnish...,1 shot,1,shot,1.0,25.0,NaN,NaN,NaN,ml,25,Rum 25.0 ml,Dark Rum 25.0 ml,Rum 25.0 ml


In [17]:
# Reallocate ingredients as columns so each cocktail takes only one row

df_cut = df[['strDrink', 'Sentence_type', 'Sentence_ingr', 'Sentence_type_ingr']]
s =  df_cut.groupby('strDrink').cumcount().add(1)
df_cut = (df_cut.set_index(['strDrink',s]).unstack().sort_index(axis=1, level=1))
df_cut.columns = ['{}_{}'.format(a, b) for a,b in df_cut.columns]

df_cut = df_cut.reset_index()
df_cut.head()

,strDrink,Sentence_ingr_1,Sentence_type_1,Sentence_type_ingr_1,Sentence_ingr_2,Sentence_type_2,Sentence_type_ingr_2,Sentence_ingr_3,Sentence_type_3,Sentence_type_ingr_3,...,Sentence_type_ingr_8,Sentence_ingr_9,Sentence_type_9,Sentence_type_ingr_9,Sentence_ingr_10,Sentence_type_10,Sentence_type_ingr_10,Sentence_ingr_11,Sentence_type_11,Sentence_type_ingr_11
0,'57 Chevy with a White License Plate,Creme De Cacao White 30.0 ml,Creamy Liqueur 30.0 ml,Creamy Liqueur 30.0 ml,Vodka 30.0 ml,Vodka 30.0 ml,Vodka 30.0 ml,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1-900-FUK-MEUP,Absolut Kurant 15.0 ml,Vodka 15.0 ml,Vodka 15.0 ml,Grand Marnier 7.5 ml,Triple Sec 7.5 ml,Triple Sec 7.5 ml,Chambord Raspberry Liqueur 7.5 ml,Sweet Liqueur 7.5 ml,Sweet Liqueur 7.5 ml,...,Pineapple Juice 7.5 ml,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,110 in the shade,Lager 480.0 ml,Beer 480.0 ml,Beer 480.0 ml,Tequila 45.0 ml,Tequila 45.0 ml,Tequila 45.0 ml,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,151 Florida Bushwacker,Malibu Rum 15.0 ml,Rum 15.0 ml,Rum 15.0 ml,Light Rum 15.0 ml,Rum 15.0 ml,Rum 15.0 ml,151 Proof Rum 15.0 ml,Rum 15.0 ml,Rum 15.0 ml,...,Vanilla Ice-Cream 128.0 gr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,155 Belmont,Dark Rum 25.0 ml,Rum 25.0 ml,Rum 25.0 ml,Light Rum 50.0 ml,Rum 50.0 ml,Rum 50.0 ml,Vodka 25.0 ml,Vodka 25.0 ml,Vodka 25.0 ml,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
def combine(line, col):
    lst1 = [line[col+'_'+str(i)] for i in range(1, 12)]
    lst2 = [str(i) for i in lst1 if str(i) != 'nan']
    return ' '.join(lst2)

        
# combine(df_cut.iloc[1], 'Sentence_ingr')      # test

In [43]:
cols = ['Sentence_ingr', 'Sentence_type', 'Sentence_type_ingr']
for i in cols:
    df_cut[i] = df_cut.apply(lambda x: combine(x, i), axis=1)

In [45]:
df_cut = df_cut[['strDrink', 'Sentence_ingr', 'Sentence_type', 'Sentence_type_ingr']]
df_cut.head()

,strDrink,Sentence_ingr,Sentence_type,Sentence_type_ingr
0,'57 Chevy with a White License Plate,Creme De Cacao White 30.0 ml Vodka 30.0 ml,Creamy Liqueur 30.0 ml Vodka 30.0 ml,Creamy Liqueur 30.0 ml Vodka 30.0 ml
1,1-900-FUK-MEUP,Absolut Kurant 15.0 ml Grand Marnier 7.5 ml Ch...,Vodka 15.0 ml Triple Sec 7.5 ml Sweet Liqueur ...,Vodka 15.0 ml Triple Sec 7.5 ml Sweet Liqueur ...
2,110 in the shade,Lager 480.0 ml Tequila 45.0 ml,Beer 480.0 ml Tequila 45.0 ml,Beer 480.0 ml Tequila 45.0 ml
3,151 Florida Bushwacker,Malibu Rum 15.0 ml Light Rum 15.0 ml 151 Proof...,Rum 15.0 ml Rum 15.0 ml Rum 15.0 ml Creamy Liq...,Rum 15.0 ml Rum 15.0 ml Rum 15.0 ml Creamy Liq...
4,155 Belmont,Dark Rum 25.0 ml Light Rum 50.0 ml Vodka 25.0 ...,Rum 25.0 ml Rum 50.0 ml Vodka 25.0 ml sweet 25...,Rum 25.0 ml Rum 50.0 ml Vodka 25.0 ml Orange J...
